Reference: https://towardsdatascience.com/web-scraping-indeed-com-e5591790736d

Reference:
https://github.com/rowandl/portfolio/blob/master/Webscraping%20Indeed.com/Webscraping%20Indeed.ipynb

In [7]:
#!pip install pandas
#!pip install bs4
#!pip install matplotlib
#!pip install lxml

In [8]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import csv
import time

from IPython.display import HTML

In [16]:
# top 10 canada cities
cities = ['Toronto', 'Montreal', 'Vancouver', 'Calgary', 'Edmonton', 'Ottawa', 'Winnipeg', 'Quebec+City', 'Hamilton', 'Kitchener']
#cities = ['Toronto']

# Set this to a high-value to generate more results. High value might get block by website
max_results_per_city = 100
#max_results_per_city = 10

total_count = len(cities) * max_results_per_city
city_count = 0
job_count = 0

#making a dataframe which collects all the information from my webscraping
#https://www.indeed.ca/jobs?q=data+scientist+%2420%2C000&l=Toronto&start=0
url_template = "https://www.indeed.ca/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
base_url = "https://ca.indeed.com"

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary', 'job_description','job_url'])

for city in cities:
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, "html.parser")
        for b in soup.find_all('div', attrs = {'class': 'result'}):
            try:
                location = b.find('div', attrs = {'class': 'location'}).text
            except:
                try:
                    location = b.find('span', attrs = {'class': 'location'}).text
                except:
                    location = 'NA'

            job_title = b.find('a', attrs = {'data-tn-element':'jobTitle'}).text
            job_title = job_title.replace("\n","")
            
            job_url = b.find('a', attrs = {'data-tn-element':'jobTitle'}).get('href')

            try:
                company = b.find('span', attrs = {'class':'company'}).text
                company = company.replace("\n","")
            except:
                company = 'NA'
            try:
                salary = b.find('td', attrs = {'class' : 'snip'}).find('nobr').text  
            except:
                salary = 'NA'
            
            time.sleep(1) # to prevent web blocking

            url_job = 'https://ca.indeed.com' + job_url
            html_job = urllib.request.urlopen(url_job).read()
            soup_job = BeautifulSoup(html_job, "html.parser")

            try:
                description = soup_job.find('div', attrs = {'id': 'jobDescriptionText'}).text
            except:
                description = 'NA'

            done = int((city_count*100+job_count)/total_count*100)

            print("%d%%: %s: %s > %s > %s" % (done, city, location, company, job_title))

            if (description != 'NA'):
                df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary, "job_description": description, "job_url": url_job}, ignore_index=True)
            
            #time.sleep(1) # to prevent web blocking
            job_count += 1

    city_count += 1
    job_count = 0
   
df.drop_duplicates(inplace=True)
df.company.replace(inplace=True,regex=True,to_replace="\n",value="")
df.job_title.replace(inplace=True,regex=True,to_replace="\n",value="")
df.head()

0%: Toronto: Greater Toronto Area, ON > fgf brands > Data Scientist Co-op ( 4 or 8 Month Winter Placement)
0%: Toronto: North York, ON > Poros Career > Data Analyst
0%: Toronto: Toronto, ON > Indigo Park > Data Scientist
0%: Toronto: Toronto, ON > Sunnyfuture Group > Data Analyst/Financial Analyst
0%: Toronto: Toronto, ON > TELUS > Data Scientist
0%: Toronto: Toronto, ON > QuadReal > Intern, Data Scientist
0%: Toronto: Toronto, ON > Overbond > Data Scientist
0%: Toronto: Toronto, ON > Equifax > Data Scientist - Client Success
0%: Toronto: Toronto, ON > Gannett > Data Scientist
0%: Toronto: Toronto, ON > TELUS > Data Scientist - TELUS Digital
1%: Toronto: Mississauga, ON > Compass Digital Labs > Data Scientist (Decision Science)
1%: Toronto: Mississauga, ON > Hatch > Data Scientist
1%: Toronto: Toronto, ON > float simple > Lead Data Scientist - BI Analytics
1%: Toronto: Toronto, ON > 7D Surgical > Data Analyst Intern
1%: Toronto: Mississauga, ON > Traffic Tech Inc. > Data Scientist
1%: 

,location,company,job_title,salary,job_description,job_url
0,"Greater Toronto Area, ON",fgf brands,Data Scientist Co-op ( 4 or 8 Month Winter Pla...,NA,Job Description:\n\nWe’re a naan traditional c...,https://ca.indeed.com/rc/clk?jk=7bf34b733f5009...
1,"North York, ON",Poros Career,Data Analyst,NA,Job Requirement:Having an degree in Mathematic...,https://ca.indeed.com/company/Poros-Career/job...
2,"Toronto, ON",Indigo Park,Data Scientist,NA,Indigo Park is an industry leader and the most...,https://ca.indeed.com/company/Indigo-Park/jobs...
3,"Toronto, ON",Sunnyfuture Group,Data Analyst/Financial Analyst,NA,"We are looking for a passionate, creative indi...",https://ca.indeed.com/company/SunnyFuture-Care...
4,"Toronto, ON",TELUS,Data Scientist,NA,"Data Scientist\nToday, TELUS business customer...",https://ca.indeed.com/rc/clk?jk=bbe976417dee58...


In [17]:
from datetime import datetime

# sending it to csvs to save the data
now = datetime.now()
dt_string = now.strftime("%y%m%d%H%M")
filename = "Indeed_" + dt_string + ".csv"
df.to_csv(filename , sep=',', encoding='utf-8')

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')
%cp *.csv /content/gdrive/My\ Drive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>